# Imports

In [9]:
%load_ext autoreload
%autoreload 2

In [10]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import pickle
import pandas as pd
pd.set_option('display.max_columns', 200)

from card import Card
from board import Board
from player import Player
from rule import Rule
import optimal_strategy as opt

import multiprocessing
import time
from multiprocessing.pool import ThreadPool
import threading
from tqdm import notebook
notebook.tqdm.pandas()

from euchre_lib import *
from search_lib import *

In [11]:
notebook.tqdm.get_lock().locks = []

# Create dataset

In [125]:
mx_thresh = 72; N_CPUS = 8
thresholds = [mx_thresh]*N_CPUS
n_hands = int(1e4)
n_epochs = 100
# get 8 million = 8e6 = 8 CPUS x 1e4 hands x 1e2 epochs
# estimate it takes 35 minutes, given 500 hands/sec

In [ ]:
parallel_search_wrapper(thresholds, MAX_CPUS=N_CPUS, n_epochs=n_epochs, n_hands=n_hands,\
                        opp_thresh=mx_thresh, title='Making dataset', folder='RLdataset')

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Making dataset:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
#def parallel_search_wrapper(thresholds, MAX_CPUS=8, n_epochs=10, n_hands=100, opp_thresh=None,
#                            title=None, ROOT_DIR=os.getcwd()+'/', folder=None):

# Predict Expert Moves

In [49]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

2.4.0


In [50]:
infolder = os.path.join(os.getcwd(), 'thresholds', 'thresh70')
data = read_all_hands(infolder)
data.shape

  0%|          | 0/100 [00:00<?, ?it/s]

(1000000, 122)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 122 entries, p0c1 to p3trueid
dtypes: int64(101), object(21)
memory usage: 930.8+ MB


In [11]:
data.head()

,p0c1,p0c1isD,p0c1isH,p0c1isS,p0c2,p0c2isD,p0c2isH,p0c2isS,p0c3,p0c3isD,p0c3isH,p0c3isS,p0c4,p0c4isD,p0c4isH,p0c4isS,p0c5,p0c5isD,p0c5isH,p0c5isS,p1c1,p1c1isD,p1c1isH,p1c1isS,p1c2,p1c2isD,p1c2isH,p1c2isS,p1c3,p1c3isD,p1c3isH,p1c3isS,p1c4,p1c4isD,p1c4isH,p1c4isS,p1c5,p1c5isD,p1c5isH,p1c5isS,p2c1,p2c1isD,p2c1isH,p2c1isS,p2c2,p2c2isD,p2c2isH,p2c2isS,p2c3,p2c3isD,p2c3isH,p2c3isS,p2c4,p2c4isD,p2c4isH,p2c4isS,p2c5,p2c5isD,p2c5isH,p2c5isS,p3c1,p3c1isD,p3c1isH,p3c1isS,p3c2,p3c2isD,p3c2isH,p3c2isS,p3c3,p3c3isD,p3c3isH,p3c3isS,p3c4,p3c4isD,p3c4isH,p3c4isS,p3c5,p3c5isD,p3c5isH,p3c5isS,TC_power,TC_isD,TC_isH,TC_isS,trump_isD,trump_isH,trump_isS,caller,round,alone,winner1,winner2,winner3,winner4,winner5,points02,points13,played1,played2,played3,played4,played5,played6,played7,played8,played9,played10,played11,played12,played13,played14,played15,played16,played17,played18,played19,played20,result,p0trueid,p1trueid,p2trueid,p3trueid
0,20,0,1,0,12,0,1,0,10,0,0,0,1,0,0,0,2,0,0,1,35,0,1,0,31,1,0,0,4,0,0,0,3,0,0,0,1,0,0,1,10,1,0,0,4,1,0,0,1,1,0,0,5,0,0,1,4,0,0,1,30,0,1,0,5,1,0,0,2,1,0,0,10,0,0,1,3,0,0,1,15,0,1,0,0,1,0,0,1,0,0,0,3,0,3,1,0,JH0,9D1,AH2,9H3,JD0,QD1,TD2,TH3,QC0,QS1,JS2,AC3,9C3,JC0,KS1,KD2,9S0,AD1,AS2,QH3,Single,3,0,1,2
1,30,0,0,0,15,0,0,0,4,0,1,0,2,0,1,0,10,0,0,1,20,0,0,0,5,1,0,0,2,1,0,0,5,0,0,1,4,0,0,1,31,0,0,1,10,1,0,0,3,1,0,0,5,0,1,0,2,0,0,1,35,0,0,0,12,0,0,0,1,1,0,0,3,0,1,0,1,0,1,0,25,0,0,0,0,0,0,1,1,0,0,3,3,3,3,0,1,JC0,TC1,QC2,KC3,JH0,QH1,TD2,KH3,AD3,9D0,TH1,KD2,JS3,9C0,AC1,QS2,JD3,9H0,AS1,KS2,Single,0,1,2,3
2,3,0,0,0,5,0,1,0,10,0,0,1,5,0,0,1,1,0,0,1,30,1,0,0,25,1,0,0,20,1,0,0,12,1,0,0,2,0,1,0,31,0,1,0,10,0,0,0,2,0,0,0,4,0,1,0,2,0,0,1,35,1,0,0,4,0,0,0,1,0,0,0,1,0,1,0,3,0,0,1,15,1,0,0,1,0,0,0,1,0,2,0,0,0,0,2,0,AD0,JH1,JD2,TD3,QC2,JC3,9D0,TC1,KD0,TS1,9C2,KH3,QD0,QH1,9H2,KS3,TH0,AC1,JS2,AS3,Sweep,1,2,3,0
3,35,0,0,0,30,0,0,0,12,0,0,0,10,1,0,0,1,0,1,0,5,1,0,0,4,1,0,0,10,0,1,0,4,0,1,0,10,0,0,1,15,0,0,0,2,1,0,0,3,0,1,0,5,0,0,1,4,0,0,1,31,0,0,1,3,1,0,0,5,0,1,0,2,0,1,0,2,0,0,1,25,0,0,0,0,0,0,1,1,0,2,1,1,1,1,0,1,KH0,9H1,AH2,JH3,AS2,QS3,TS0,9C1,JC1,QD2,TC3,JS0,AD1,KD2,KS3,JD0,AC1,QH2,KC3,TH0,Single,2,3,0,1
4,10,0,0,0,4,0,0,0,10,1,0,0,2,0,1,0,1,0,1,0,5,1,0,0,4,1,0,0,1,1,0,0,5,0,1,0,4,0,1,0,31,0,0,0,25,0,0,1,2,0,0,0,1,0,0,0,3,0,1,0,30,0,0,1,20,0,0,1,12,0,0,1,5,0,0,0,2,1,0,0,35,0,0,1,0,0,1,1,1,0,1,2,3,3,2,0,1,KD0,KS1,TD2,AD3,JH1,9S2,TH3,QH0,KC2,AC3,9D0,9C1,JS3,QD0,JC1,QS2,QC3,KH0,TC1,AS2,Single,3,0,1,2


In [32]:
np.argwhere(np.isin(np.array([v[:-1] for v in data.iloc[0][['played'+str(i) for i in range(1, 21)]]]), 'KH')).reshape(-1)

array([], dtype=int64)

In [54]:
p0hand = ['JH', 'JD', 'QC', 'JC', '9S']
p1hand = ['AD', 'QD', '9D', 'KS', 'QS']
p2hand = ['AH', 'KD', 'TD', 'AS', 'JS']
p3hand = ['QH', '9H', 'AC', '9C', 'TS'] + ['TH']

In [101]:
COLS_GROUP1 = 24
COLS_GROUP2 = 47
COLS_GROUP3 = 24*13
COLS_GROUP4 = 55
COLS_TOTAL = COLS_GROUP1 + COLS_GROUP2 + COLS_GROUP3 + COLS_GROUP4
same_color_suit = {'C':'S', 'D':'H', 'H':'D', 'S':'C'}
def format_data(data, usetqdm=True, start=0, stop=None, count=None):
    """
    Here is all the data that needs to be fed to the ML algorithm, grouped by phase of the game.
    I have also tried to include an estimate of how many columns each will need to take up.
    If a categorical feature has N options, I will OHE it as N columns, instead of using N-1.
    A card will be OHEncoded as [9-A] + [C/D/H/S] (6+4), and possibly tagged as Y/N trump.
    
    #######
    DATA GROUP 1: Calling trump
    #######
    (4)  1.) Who dealt (position relative to "me")
    (4)  2.) Who called trump (position relative to "me")
    (1)  3.) Which round was trump called in
    (1)  4.) Going alone?
    (4)  5.) Which suit is trump (not sure about this one)
    (10) 6.) What is the turn card
    Total: 24 columns
    
    #######
    DATA GROUP 2: Other misc. information
    #######
    (4)  1.) Who is leading right now
    (4)  2.) Who is winning right now
    (11) 3.) What card was led (is it trump)
    (11) 4.) What card is winning (is it trump)
    (5)  5.) Which team won each trick so far (+1 for "me", 0 for no one (yet), -1 for opponents)
    (12) 6.) Any players confirmed short in any suits
    Total: 47 columns
    
    #######
    DATA GROUP 3: All card locations (constant order: 9C, 10C, ..., (D), (H), ..., KS, AS)
    #######
    For each card (24):
    (4)  1.) Confirmed in anyone's hand (my hand + ordered up turn card?)
    (4)  2.) Played in a previous trick by someone (maybe later expand this to which prev trick?)
    (3)  3.) Played in CURRENT trick by someone
    (1)  4.) Is trump
    Total: 288 columns
    
    #######
    DATA GROUP 4: My remaining hand, again
    #######
    (11) 1.) Card #1 (is it trump)
    (11) 2.) Card #2 (is it trump)
    (11) 3.) Card #3 (is it trump)
    (11) 4.) Card #4 (is it trump)
    (11) 5.) Card #5 (is it trump)
    Total: 55 columns
    
    SUPER-TOTAL: 414 columns. Yeesh.
    """
    
    counter = 0
    stop = len(data) if stop is None else stop
    count = len(data) if count is None else count
    formatted = np.zeros((20*(stop-start), COLS_TOTAL), dtype=np.int8)
    
    for i in notebook.tqdm(data.index) if usetqdm else data.index:
        i = int(i)
        if i < start:  continue
        elif i >= stop: break
        elif counter >= count: break
        
        game = data.iloc[i]
        formatted[20*counter:20*(counter+1)] = format_game(game)
        counter += 1
    return formatted[~np.all(formatted==0, axis=1)]

In [108]:
def format_game(game):
    formatted_game = np.zeros((20, COLS_TOTAL), dtype=np.int8)
    
    for tricknum in range(5):
        for playernum in range(4):
            group1_info = get_group1_info(game, tricknum, playernum)
            group2_info = get_group2_info(game, tricknum, playernum)
            group3_info = get_group3_info(game, tricknum, playernum)
            group4_info = get_group4_info(game, tricknum, playernum)
            
            formatted_game[4*tricknum+playernum, :len(group1_info)] = group1_info
            formatted_game[4*tricknum+playernum, len(group1_info):len(group1_info)+len(group2_info)] = group2_info
            formatted_game[4*tricknum+playernum, len(group1_info)+len(group2_info):\
                           len(group1_info)+len(group2_info)+len(group3_info)] = group3_info
            formatted_game[4*tricknum+playernum, len(group1_info)+len(group2_info)+len(group3_info):\
                           len(group1_info)+len(group2_info)+len(group3_info)+len(group4_info)] = group4_info
    return formatted_game

In [86]:
def get_group1_info(game, tricknum, playernum):
    """
    #######
    DATA GROUP 1: Calling trump
    #######
    (4)  1.) Who dealt (position relative to "me")
    (4)  2.) Who called trump (position relative to "me")
    (1)  3.) Which round was trump called in
    (1)  4.) Going alone?
    (4)  5.) Which suit is trump (not sure about this one)
    (10) 6.) What is the turn card
    Total: 24 columns
    """
    group1_info = np.zeros(COLS_GROUP1, dtype=np.int8)
    current_player = get_current_player(game, tricknum, playernum)
    
    # who dealt
    group1_info[get_relative_position(game, tricknum, playernum, '3')] = 1
    # who called
    group1_info[4+get_relative_position(game, tricknum, playernum, game['caller'])] = 1
    # was it called first round
    group1_info[8] = 2-int(game['round'])
    # did they go alone
    group1_info[9] = int(game['alone'])
    # which suit is trump
    group1_info[10+{'C':0, 'D':1, 'H':2, 'S':3}[get_trump_suit(game)]] = 1
    # what is the turn card
    turn_card = get_turn_card(game)
    group1_info[14+{n:i for n,i in zip(list('9TJQKA'), range(6))}[turn_card[0]]] = 1
    group1_info[20+{s:i for s,i in zip(list('CDHS'), range(4))}[turn_card[1]]] = 1
    
    return group1_info

In [112]:
def get_winner(current_trick, trump_suit):
    winning_card = current_trick[0]
    powers = {n+s:p for n,p in zip(list('9TJQKA'), [1,2,3,4,5,10]) for s in list('CDHS') if s != trump_suit}
    powers['J'+same_color_suit[trump_suit]] = 31
    powers.update({n+trump_suit:p for n,p in zip(list('9TQKAJ'), [12,15,20,25,30,35])})
    
    for i in range(1,len(current_trick)):
        c = current_trick[i]
        # if winning card is trump, just compare powers
        if winning_card[1] == trump_suit or (winning_card[0]=='J' and winning_card[1]==same_color_suit[trump_suit]):
            if powers[c[:2]] > powers[winning_card[:2]]:
                winning_card = c
        else:
            # first, check if card is trump
            if powers[c[:2]] > 10:
                winning_card = c
            # next, check if some random offsuit
            elif c[1] != winning_card[1]:
                continue
            # by now, determined neither are trump, and both have the same suit
            else:
                if powers[c[:2]] > powers[winning_card[:2]]:
                    winning_card = c
    return int(winning_card[-1]), winning_card[:2]

In [88]:
def get_short_suitedness(game, tricknum, playernum, opp_pos, short_suit):
    led_cards = [c for c in game[['played'+str(i+1) for i in range(4*tricknum+playernum)]][::4]]
    trump_suit = get_trump_suit(game)
    
    for i, c in enumerate(led_cards):
        # skip if they lead
        if get_relative_position(game, tricknum, playernum, c[-1]) == opp_pos:
            continue
        
        associated_trick = game[['played'+str(i+1) for i in range(4*i, min(4*(i+1), 4*tricknum+playernum))]]
        # skip if they haven't played yet
        if opp_pos not in [get_relative_position(game, tricknum, playernum, c[-1]) for c in associated_trick]:
            continue
        opp_played_card = [c for c in associated_trick if get_relative_position(game, tricknum, playernum, c[-1])==opp_pos][0]
        
        if c[1] == trump_suit or (c[0]=='J' and c[1] == same_color_suit[trump_suit]):
            # "if not trump suit and also is not left"
            if opp_played_card[1] != trump_suit and not (opp_played_card[0]=='J' and opp_played_card[1]==same_color_suit[trump_suit]):
                return 1
        else:
            # "if not same suit or is left"
            if opp_played_card[1] != c[1] or (opp_played_card[0]=='J' and opp_played_card[1]==same_color_suit[trump_suit]):
                return 1
    return 0

In [110]:
def get_group2_info(game, tricknum, playernum):
    """
    #######
    DATA GROUP 2: Other misc. information
    #######
    (4)  1.) Who is leading right now
    (4)  2.) Who is winning right now
    (11) 3.) What card was led (is it trump)
    (11) 4.) What card is winning (is it trump)
    (5)  5.) Which team won each trick so far (+1 for "me", 0 for no one (yet), -1 for opponents)
    (12) 6.) Any players confirmed short in any suits
    Total: 47 columns
    """
    group2_info = np.zeros(COLS_GROUP2, dtype=np.int8)
    current_trick = game[['played'+str(i+1) for i in range(4*tricknum, 4*tricknum+playernum)]]
    trump_suit = get_trump_suit(game)
    
    # who leads
    group2_info[get_relative_position(game, tricknum, playernum, current_trick[0][-1]) if len(current_trick) > 0 else 3] = 1
    # who's winning
    if len(current_trick) > 0:
        winner, winning_card = get_winner(current_trick, trump_suit)
        group2_info[get_relative_position(game, tricknum, playernum, winner)] = 1
    # what card was led
    if len(current_trick) > 0:
        group2_info[8+{n:i for n,i in zip(list('9TJQKA'), range(6))}[current_trick[0][0]]] = 1
        group2_info[14+{s:i for s,i in zip(list('CDHS'), range(4))}[current_trick[0][1]]] = 1
        group2_info[18] = (current_trick[0][1]==trump_suit) or (current_trick[0][0]=='J' and current_trick[0][1]==same_color_suit[trump_suit])
    # what card is winning
    if len(current_trick) > 0:
        group2_info[19+{n:i for n,i in zip(list('9TJQKA'), range(6))}[winning_card[0]]]
        group2_info[25+{s:i for s,i in zip(list('CDHS'), range(4))}[winning_card[1]]] = 1
        group2_info[29] = (winning_card[1]==trump_suit) or (winning_card[0]=='J' and winning_card[1]==same_color_suit[trump_suit])
    # what team won each trick so far
    for tnum in range(5):
        if tnum >= tricknum:
            continue
        # return +1 if relative_position % 2 == 1, return -1 if relative_position % 2 == 0 (self is always 3)
        group2_info[30+tnum] = -1+2*(get_relative_position(game, tricknum, playernum, game['winner'+str(tnum+1)])%2)
    # any players confirmed short in suits
    # list it like [opp1 short in clubs, opp1 short in diamonds, ..., opp2 short in spades]
    for opp_pos in range(3):
        for i, s in enumerate(list('CDHS')):
            group2_info[35+4*opp_pos + i] = get_short_suitedness(game, playernum, tricknum, opp_pos, s)

    return group2_info

In [90]:
def get_current_player(game, tricknum, playernum):
    return game['played'+str(4*tricknum+playernum+1)][-1]

In [91]:
def get_relative_position(game, tricknum, playernum, pos):
    return (int(pos) - int(get_current_player(game, tricknum, playernum)) - 1)%4
    # self maps to 3, then advances positively by advancing other player pos

In [92]:
power_to_name = {power:n for power,n in zip([1,2,3,4,5,10,12,15,20,25,30,31,35], list('9TJQKA9TQKAJJ'))}
def get_turn_card(game):
    name = power_to_name[game['TC_power']]
    if game['TC_isD']: return name+'D'
    elif game['TC_isH']: return name+'H'
    elif game['TC_isS']: return name+'S'
    else: return name+'C'

In [93]:
def get_original_hand(game, tricknum, playernum):
    player = get_current_player(game, tricknum, playernum)
    return [power_to_name[game['p'+str(player)+'c'+str(i+1)]] + 
            'D'*game['p'+str(player)+'c'+str(i+1)+'isD'] + \
            'H'*game['p'+str(player)+'c'+str(i+1)+'isH'] + \
            'S'*game['p'+str(player)+'c'+str(i+1)+'isS'] + \
                'C'*(1-game['p'+str(player)+'c'+str(i+1)+'isD']-\
                     game['p'+str(player)+'c'+str(i+1)+'isH']-\
                     game['p'+str(player)+'c'+str(i+1)+'isS'])\
            for i in range(5)]

In [94]:
card_ix = {**{n:i for n,i in zip(list('9TJQKA'), range(6))},\
            **{s:6*i for s,i in zip(list('CDHS'), range(4))}}
def get_group3_info(game, tricknum, playernum):
    """
    #######
    DATA GROUP 3: All card locations (constant order: 9C, 10C, ..., (D), (H), ..., KS, AS)
    #######
    For each card (24):
    (4)  1.) Confirmed in anyone's hand (my hand + ordered up turn card?)
    (4)  2.) Played in a previous trick by someone (maybe later expand this to which prev trick?)
    (3)  3.) Played in CURRENT trick by someone
    (1)  4.) Confirmed buried
    (1)  5.) Is trump
    Total: 312 columns
    """
    COLS_PER_CARD = 12
    group3_info = np.zeros(24*COLS_PER_CARD, dtype=np.int8)
    trump_suit = get_trump_suit(game)
    
    # cards played in a previous trick
    if tricknum > 0:
        prev_played_cards = game[['played'+str(i+1) for i in range(4*tricknum)]]
        for c in prev_played_cards:
            group3_info[COLS_PER_CARD*(card_ix[c[0]] + card_ix[c[1]]) + 4 + get_relative_position(game, tricknum, playernum, c[-1])] = 1
    
    # cards played THIS trick
    if playernum > 0:
        current_played_cards = game[['played'+str(i+1) for i in range(4*tricknum, 4*tricknum+playernum)]]
        for c in current_played_cards:
            group3_info[COLS_PER_CARD*(card_ix[c[0]] + card_ix[c[1]]) + get_relative_position(game, tricknum, playernum, c[-1])] = 1
    
    # cards in my hand
    my_remaining_cards = [c[:-1] for c in game[['played'+str(i+1) for i in range(4*tricknum+playernum, 20)]]\
                    if c[-1] == get_current_player(game, tricknum, playernum)]
    for c in my_remaining_cards:
        # position of self wrt self is always 3
        group3_info[COLS_PER_CARD*(card_ix[c[0]] + card_ix[c[1]]) + 3] = 1
        
    # confirmed turn card location
    if game['round']==2:
        turn_card = get_turn_card(game)
        group3_info[COLS_PER_CARD*(card_ix[turn_card[0]] + card_ix[turn_card[1]]) + COLS_PER_CARD-2] = 1
    elif get_current_player(game, tricknum, playernum) == '3':
        original_cards = get_original_hand(game, tricknum, playernum)
        played_cards = [c[:-1] for c in game[['played'+str(i+1) for i in range(20)]] if c[-1]=='3']
        buried_card = [c for c in original_cards if c not in played_cards][0]
        group3_info[COLS_PER_CARD*(card_ix[buried_card[0]]+card_ix[buried_card[1]]) + COLS_PER_CARD-2] = 1
    else:
        turn_card = get_turn_card(game)
        all_played_cards = game[['played'+str(i+1) for i in range(4*tricknum+playernum)]]
        if turn_card+'3' not in all_played_cards:
            group3_info[COLS_PER_CARD*(card_ix[turn_card[0]]+card_ix[turn_card[1]]) + get_relative_position(game, tricknum, playernum, 3)] = 1
    
    # Mark trump
    for s in list('CDHS'):
        if s == trump_suit:
            for name in list('9TJQKA'):
                group3_info[COLS_PER_CARD*(card_ix[name]+card_ix[s]) + COLS_PER_CARD-1] = 1
            group3_info[COLS_PER_CARD*(card_ix['J']+card_ix[same_color_suit[s]]) + COLS_PER_CARD-1] = 1
        
    return group3_info

In [106]:
def get_group4_info(game, tricknum, playernum):
    """
    #######
    DATA GROUP 4: My remaining hand, again
    #######
    (11) 1.) Card #1 (is it trump)
    (11) 2.) Card #2 (is it trump)
    (11) 3.) Card #3 (is it trump)
    (11) 4.) Card #4 (is it trump)
    (11) 5.) Card #5 (is it trump)
    Total: 55 columns
    """
    """
    my_cards = [c for c in game[['played'+str(i) for i in range(1,21)]] if c[-1] == str(playernum)]
    trump_suit = get_trump_suit(game)
    np.random.shuffle(my_cards)
    my_cards = [c[:-1] if c not in game[['played'+str(i) for i in range(1,4*tricknum+playernum+1)]] else '00' for c in my_cards]
    """
    # slightly more efficient
    trump_suit = get_trump_suit(game)
    my_cards = [c[:-1] for c in game[['played'+str(i+1) for i in range(4*tricknum+playernum, 20)]]\
                if c[-1] == get_current_player(game, tricknum, playernum)]
    my_cards += ['00']*(5-len(my_cards))
    np.random.shuffle(my_cards)
    
    group4_info = []
    for c in my_cards:
        group4_info += card_to_ohe(c[0], c[1], trump_suit==c[1] or (c[0]=='J' and c[1]==same_color_suit[trump_suit]))
    return group4_info

In [56]:
get_group4_info(data.iloc[0], 0, 0)

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0]

In [96]:
def get_trump_suit(game):
    if game['trump_isD']: return 'D'
    elif game['trump_isH']: return 'H'
    elif game['trump_isS']: return 'S'
    else: return 'C'

In [97]:
def card_to_ohe(name, suit, trump=None):
    arr = [0]*10
    for i, n in enumerate(['9', 'T', 'J', 'Q', 'K', 'A']):
        if name == n:
            arr[i] = 1
            break
    for i, s in enumerate(['C', 'D', 'H', 'S']):
        if suit == s:
            arr[6+i] = 1
            break
    if trump is not None:
        arr += [int(trump)]
    return arr

In [113]:
format_data(data.iloc[:3])[0]

  0%|          | 0/3 [00:00<?, ?it/s]

array([0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [13]:
24+47+288+55

414

In [24]:
x = np.zeros((5,5))
x

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [27]:
x[1:3, 1:3] = [[1,2],[3,4]]
x

array([[0., 0., 0., 0., 0.],
       [0., 1., 2., 0., 0.],
       [0., 3., 4., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [36]:
x[~np.all(x==0, axis=1)]

array([[0., 1., 2., 0., 0.],
       [0., 3., 4., 0., 0.]])

In [44]:
import random
z = [1,23,3,4,5]
np.random.shuffle(z)
z

[3, 1, 5, 4, 23]